# Part 2: Fixing issues in our Data - part 1

... introduction  
... mention sql functions  
To achieve this we are going to be using some functions from [the pyspark.sql.functions module](https://spark.apache.org/docs/2.4.3/api/python/pyspark.sql.html#module-pyspark.sql.functions).  
*__TIP:__ Take your time to study the `functions` module, as it contains a lot of useful functions.*

... that functions need to be explicitly imported, as they are otherwise not available for use

... Same as in part 1, we are going to be using the `ratings.csv` as provided in the MovieLens data. 

## SparkSession and Settings
Before we continue, set up a SparkSession.

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("MovieLens_Tutorial").getOrCreate()

Additionally, we have to define some settings to ensure proper operations.
 - `RATINGS_CSV_LOCATION` is used to tell our Spark Application where to find the ratings.csv file

In [ ]:
# Location of the ratings.csv  file
RATINGS_CSV_LOCATION = "/home/jovyan/data/ml-latest-small/ratings.csv"

## Loading Data
Before proceeding, ensure that the CSV data is loaded to a proper `df` object (output of part 1)

In [ ]:
#  Type safe loading of ratings.csv file
df = spark.read.csv(
    path=RATINGS_CSV_LOCATION,
    sep=",",
    header=True,
    quote='"',
    encoding="UTF-8",
    schema="userId INT, movieId INT, rating DOUBLE, timestamp INT",
)

## Unix Timestamps

The `timestamp` column in the DataFrame we previously loaded does not seem to be parsed well yet, as it is not very human readable. A value in this timestamp column looks something like this `964982703`.

`README.txt` belonging to the MovieLens data says this about the timestamp column:
> Timestamps represent seconds since midnight Coordinated Universal Time (UTC) of January 1, 1970.

These type of timestamps are called [__POSIX time__ or __UNIX Epoch time__](https://en.wikipedia.org/wiki/Unix_time).


## Converting Timestamps using SQL Functions
Let's set out to convert this Unix timestamp to a proper, human-readable, `TimeStampType()` value using some of the functions available to use in the functions module.

We are going to need the following functions to allow us to do this:

 - We need [the `col()` function](https://spark.apache.org/docs/2.4.3/api/python/pyspark.sql.html#pyspark.sql.functions.col):
 This allows us to reference columns by their name.
 - We need [the `from_unixtime()` function](https://spark.apache.org/docs/2.4.3/api/python/pyspark.sql.html#pyspark.sql.functions.from_unixtime):
 This allows us to convert the UNIX Epoch time to a `StringType`.  
 - We need [the `to_timestamp()` function](https://spark.apache.org/docs/2.4.3/api/python/pyspark.sql.html#pyspark.sql.functions.to_timestamp):
 This allow us to convert from `StringType` to a `TimestampType`. 
 
So let's import these into our Spark Application.

In [ ]:
from pyspark.sql.functions import col, from_unixtime, to_timestamp

Since we are going to want to format our Timestamp in a certain format, we have to tell the system what format we want to use.  
*__TIP__: A good reference for possible formats for dates and times can be found here: https://docs.oracle.com/javase/8/docs/api/java/text/SimpleDateFormat.html*


In [ ]:
# Default ISO8601 format for dates
DATEFORMAT = "yyyy-MM-dd'T'HH:mm:ssZ"

Overall plan of approach:
1. Rename original `timestamp` column to one named `timestamp_unix` using [`.withColumnRenamed` DataFrame operation](https://spark.apache.org/docs/2.4.3/api/python/pyspark.sql.html#pyspark.sql.DataFrame.withColumnRenamed)
2. We regenerate a `timestamp` column using [`.withColumn` DataFrame operation](https://spark.apache.org/docs/2.4.3/api/python/pyspark.sql.html#pyspark.sql.DataFrame.withColumn), and fill it using the `from_unixtime` to transform the unix timestamp to a string with the given format 
   *We defined the format of our timestamp in the `DATEFORMAT` variable previously*
3. We overwrite the `timestamp` column again using [`.withColumn` DataFrame operation](https://spark.apache.org/docs/2.4.3/api/python/pyspark.sql.html#pyspark.sql.DataFrame.withColumn), converting the previously generated string to a TimeStampType using `to_timestamp`

Let's start with the first two steps and see what our data then looks like

In [ ]:
# Rename timestamp to timestamp_unix
df = df.withColumnRenamed("timestamp", "timestamp_unix")

# Converting UNIX time to String with the given DATEFORMAT
df = df.withColumn("timestamp", from_unixtime(col("timestamp_unix"), DATEFORMAT))

# Displaying results of the transformation
df.show()
df.printSchema()

Our timestamp has now been converted to a string with a nice format  
> `|-- timestamp: string (nullable = true)`

Lets convert it to a TimestampType with the `to_timestamp` function and see the result.

In [ ]:
# Convert timestamp string with the given DATEFORMAT to TimeStampType()
df = df.withColumn("timestamp", to_timestamp(col("timestamp"), DATEFORMAT))

# Let's see our output
df.show()
df.printSchema()

Now, our data looks the way it should. The unixtime is correctly converted to a timestamp value.  
In the schema you can now see `|-- timestamp: timestamp (nullable = true)`

## Dropping columns

For good measure, let's drop the `timestamp_unix` column since we will not be needing it anymore.  
We can do this using [the `.drop` method](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.drop) on our DataFrame.

In [ ]:
# Drop redundant column
df = df.drop(col('timestamp_unix'))

# Let's see our output
df.show()
df.printSchema()

## Chaining Operations
... a note about chaining operations  
... so far every operation has been on a newline, this is not only way of doing it

In [ ]:
# Chained operation, from loading to cleansing, all in a single operation
df = spark.read.csv(
    RATINGS_CSV_LOCATION,
    sep=",",
    header=True,
    quote='"',
    schema="userId INT, movieId INT, rating DOUBLE, timestamp INT",
).withColumn("timestamp", to_timestamp(from_unixtime(col("timestamp"))))

df.show()
df.printSchema()

## What we've learned so far:
- how to convert between strings and timestamps using the SparkSQL `functions` module
- that `functions` need to be explicitly imported, as they are otherwise not available for use
- how to rename, add, manipulate, and drop columns using the `.withColumn`, `withColumnRenamed`, and `.drop()` methods
- how operations can be efficiently chained...

---

In [ ]:
# Stop the spark application, frees up resources
spark.stop()

end of part 2